In [ ]:
import os
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
import seaborn as sns

base_path = "../data/raw/tennis_data"
days_to_query = os.listdir(base_path)

home_score_files = []
away_score_files = []
tournament_files = []
home_player_files = []
away_player_files = []

for day in days_to_query:
    day_path = os.path.join(base_path, day)
    home_score_files.extend(glob(f"{day_path}/data/raw/raw_match*/home_team_score_*.parquet"))
    away_score_files.extend(glob(f"{day_path}/data/raw/raw_match*/away_team_score_*.parquet"))
    tournament_files.extend(glob(f"{day_path}/data/raw/raw_match*/tournament_*.parquet"))
    home_player_files.extend(glob(f"{day_path}/data/raw/raw_match*/home_team_1*.parquet"))
    away_player_files.extend(glob(f"{day_path}/data/raw/raw_match*/away_team_1*.parquet"))

def load_and_tag(files):
    frames = []
    for path in files:
        try:
            date_str = path.split("/")[4]
            df = pd.read_parquet(path)
            df["match_date"] = pd.to_datetime(date_str, format="%Y%m%d")
            frames.append(df)
        except:
            continue
    return pd.concat(frames, ignore_index=True)

home_scores = load_and_tag(home_score_files)
away_scores = load_and_tag(away_score_files)
tournament_info = load_and_tag(tournament_files)
home_players = load_and_tag(home_player_files)
away_players = load_and_tag(away_player_files)

scores = pd.merge(home_scores, away_scores, on="match_id", suffixes=("_home", "_away"))

set_cols = [1, 2, 3, 4, 5]
for i in set_cols:
    scores[f"set_{i}_winner"] = scores.apply(
        lambda row: "home" if row[f"period_{i}_home"] > row[f"period_{i}_away"]
        else ("away" if row[f"period_{i}_home"] < row[f"period_{i}_away"] else "tie"),
        axis=1
    )

scores["home_sets_won"] = scores[[f"set_{i}_winner" for i in set_cols]].apply(lambda r: sum(r == "home"), axis=1)
scores["away_sets_won"] = scores[[f"set_{i}_winner" for i in set_cols]].apply(lambda r: sum(r == "away"), axis=1)
scores["match_winner"] = scores.apply(
    lambda row: "home" if row["home_sets_won"] > row["away_sets_won"] else "away", axis=1
)

home_players = home_players.rename(columns={"name": "home_player"})
away_players = away_players.rename(columns={"name": "away_player"})

scores = scores.merge(home_players[["match_id", "home_player"]], on="match_id", how="left")
scores = scores.merge(away_players[["match_id", "away_player"]], on="match_id", how="left")

scores["winner_name"] = scores.apply(
    lambda row: row["home_player"] if row["match_winner"] == "home" else row["away_player"],
    axis=1
)

scores = scores.merge(tournament_info[["match_id", "tournament_unique_id"]], on="match_id", how="left")
scores["year_month"] = scores["match_date"].dt.to_period("M")

finals = scores.sort_values(["tournament_unique_id", "match_date"]).groupby("tournament_unique_id").tail(1)

win_counts = finals.groupby(["winner_name", "year_month"]).size().reset_index(name="tournaments_won")
top_winner = win_counts.sort_values("tournaments_won", ascending=False).head(1)

top10 = win_counts.sort_values("tournaments_won", ascending=False).head(10)

plt.figure(figsize=(10, 6))
sns.barplot(data=top10, x="tournaments_won", y="winner_name", hue="year_month", dodge=False, palette="viridis")
plt.title("Top 10 Player-Months by Tournaments Won")
plt.xlabel("Number of Tournaments Won")
plt.ylabel("Player")
plt.tight_layout()
plt.show()

print("🏆 Player with the most tournaments won in a single month:")
print(top_winner)